In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.llms import Ollama
import string


llm = Ollama(model="llama3", base_url="http://localhost:11434")


system_prompt = """
Você é um tutor de curso online. Responda em pt-br APENAS com base no conteúdo fornecido. 
NÃO forneça informações externas. 
Se a pergunta não estiver no contexto, diga apenas: 'Essa questão não faz parte do curso.'
"""


tools = []


agente_tutor_especializado = initialize_agent(
    tools = [],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    handle_parsing_errors=True,
    max_iterations=5,
    early_stopping_method="generate"
)

# func para normalizar texto 
def normalize_text(texto):
    texto = texto.lower()
    for p in string.punctuation:
        texto = texto.replace(p, "")
    return set(texto.split())

# func para verificar se a pergunta eh relevante para o contexto
def pergunta_relevante(pergunta, contexto):
    palavras_contexto = normalize_text(contexto)
    palavras_pergunta = normalize_text(pergunta)
    return len(palavras_contexto & palavras_pergunta) > 0

# func de teste do tutor com filtragem
def perguntar_ao_tutor(pergunta, contexto_aula):
    if not pergunta_relevante(pergunta, contexto_aula):
        return "Essa questão não faz parte do curso."
    
    prompt = f"""
    {system_prompt}

    CONTEXTO:
    {contexto_aula}

    PERGUNTA:
    {pergunta}
    """
    return agente_tutor_especializado.run(prompt)

# Contexto de exemplo
contexto = "Esta aula ensina sobre pipelines com N8N, incluindo triggers e automações básicas."

# Testes
resposta1 = perguntar_ao_tutor("Explique como pra q serve o N8N", contexto)
print("Pergunta dentro do contexto:", resposta1)

resposta2 = perguntar_ao_tutor("O que é MEM RAM?", contexto)
print("Pergunta fora do contexto:", resposta2)


Pergunta dentro do contexto: Action:
{
  "action": "Final Answer",
  "action_input": "O N8N (Node-Pipelines Next Generation) é um software de código aberto que permite criar e gerenciar pipelines automatizados para integração de diferentes aplicativos e serviços. Com o N8N, você pode criar triggers personalizados para disparar ações específicas em seu pipeline, como envio de emails ou atualização de bases de dados."
}


Pergunta fora do contexto: Essa questão não faz parte do curso.
